# 1. Preprocessing

Usually, natural language processors (NLP) refer to both speech data and textual data. There are several components necessary to understand the structure and meaning of human language.

From a linguistics perspective, it is important to look at the following:
 - syntax, the actual rules that define how words are combined to form understandable sentences;
 - semantics, referring to the meaning behind the phrases and sentences formed by syntax;
 - morphology, referring to the ways in which words can take different forms.

Then, from a computational perspective, we can take the rules from the linguistics aspects and transform linguistic knowledge into rule-based and/or ML-based algorithms to solve problems related to natural language processing.




<b>Exercise 1.1:</b> Explore the Allen NLP demos found at https://demo.allennlp.org/sentiment-analysis/glove-sentiment-analysis. 
<ol>
<li>Choose either of the two sentiment analysis algorithms and use any of the provided example sentences. Look under CLI Output. How do the algorithms take the example input sentence and transform it?</li>
<li> Look under Model Interpretations and Model Attacks, which visualize the most important words in the sentence. Which words seem to be important, and which ones are often removed? </li>
<li>Try some of your own sentences. Does the algorithm seem to be accurate?</li>
</ol>

<h2>1.1 Tasks</h2>
Within the realm of preprocessing textual data, we first must perform certain detections and tasks; this ensures that we have textual data that makes sense and is meaningful. The following all help at ensuring data cleanliness:

 - <b>Syntactic analysis:</b> Does the grammar make sense? Is the text gramatically correct?
 - <b>Semantic analysis:</b> Are we aware of the meaning of the words in context?  Do we understand the structure, word interaction, and related concepts of the text?
 - <b>Keyword extraction:</b> What are the most important words in the text?
 - <b>Named entity resolution:</b> How do we identify and extract entities (names, organizations, addresses and places) and relationships? 
 - <b>Text classification:</b> Can we organize our text into predefined categories? Ways of doing this include:
   - <b>Sentiment analysis:</b> Can we classify text (like customer feedback) into positive and negative feedback?
   - <b>Email filtering:</b> If we are taking our texts from emails, can we classify email text as spam mail and remove spam?
   - <b>Intent detection:</b> What is the text generator trying to achieve? For example, searching “apple” could indicate an intent of buying, eating, or researching apples.
   - <b>Language detection:</b> Can we classify the body of text into languages, with an associated probability?

<b>Exercise 1.2:</b> Explore another Allen NLP demo found at https://demo.allennlp.org/next-token-lm. 
<ol>
<li>Try some of the provided example sentences, and give the probabilities for the sentences with the highest score. </li>
</ol>

Looking at the UN SDG dataset, we find that it includes positive or negative labels. Since each text is classified with a single goal, positive labels indicate the agreement of that text with the goal, while negative labels indicate disagreement. We use $\texttt{sum()}$ to sum the negative labels and the positive labels:


In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [5]:
text_df = pd.read_csv("osdg-community-data-v2023-01-01.csv", sep = "\t", quotechar='"')
col_names = text_df.columns.values[0].split('\t')
text_df[col_names] = text_df[text_df.columns.values[0]].apply(lambda x: pd.Series(str(x).split("\t")))
text_df.drop(text_df.columns.values[0],axis = 1, inplace=True)
text_df = text_df.astype({'sdg':int, 'labels_negative': int, 'labels_positive':int, 'agreement': float}, copy=True)
text_df = text_df.query("agreement > 0.5 and (labels_positive - labels_negative) > 2")
#text_df["lang"] = text_df["text"].apply(lambda x: detect(x))
#text_df = text_df.query("lang == 'en'")# language detection cost extra time, do it after other filterings to minimize cost
text_df.reset_index(inplace=True)

,sdg,sdg_name,sdg_definition
0,1,No Poverty,End poverty in all its forms everywhere
1,2,Zero Hunger,"End hunger, achieve food security and improved..."
2,3,Good Health and Well Being,Ensure healthy lives and promote well-being fo...
3,4,Quality Education,Ensure inclusive and equitable quality educati...
4,5,Gender Equality,Achieve gender equality and empower all women ...
5,6,Clean Water and Sanitation,Ensure availability and sustainable management...
6,7,Affordable and Clean Energy,"Ensure access to affordable, reliable, sustain..."
7,8,Decent Work and Economic Growth,"Promote sustained, inclusive and sustainable e..."
8,9,"Industry, Innovation and Infrastructure","Build resilient infrastructure, promote inclus..."
9,10,Reduced Inequalities,Reduce inequality within and among countries


In [3]:
text_df.labels_negative.sum() + text_df.labels_positive.sum()

AttributeError: 'DataFrame' object has no attribute 'labels_negative'

You may notice that the sum of the positively and negatively labeled texts does not equal the full amount of texts imported from the dataset. Hence, this provides some beginning rationale for the necessity of preprocessing.

Data preprocessing for textual data includes some specific tasks, but it also includes some work that can be done on datasets in general. For our UN SDG dataset, it is useful to remove the columns that we don't need, which for this dataset are the columns with zeroes in all entries. We do this by the following code:

In [ ]:
text_df.drop(text_df.columns.values[0],axis = 1, inplace=True)

It is also a good idea to check our data for any discrepancies, especially N/A values and incorrect entries. 

<b>Exercise 1.3</b>: Check the data for any N/A values. Is it good to remove those entries with N/A values?

For our UN SDG dataset, we want texts that are clearly classified into a single goal. We also will want to select for only the positively labeled texts as this indicates higher agreement with the labeled goal, so our filtering will remove the rows with labeling agreement less than 0.5 and the rows where the positive labels minus the negative labels is less than 2.

In [ ]:
text_df = text_df.query("agreement > 0.5 and (labels_positive - labels_negative) > 2")
text_df.reset_index(inplace=True, drop=True)

<b>Exercise 1.4</b>: Use $\texttt{text\_df.info()}$ to examine information about the dataset after running the preprocessing in this section. What columns are included? What data types are there, and how many entries? Does this seem like a reasonable size of data to work with?

## 1.2 Levels of Processing

In (almost) any language, there are distinct levels of a text we can focus on: <b>characters</b> are strung together to form <b>words</b>, which are put together into <b>sentences</b>, then <b>paragraphs</b>, <b>documents</b>, and a whole set of documents, or a <b>corpus</b>. In all these cases, we put multiples together to make the next level, and decisions in preprocessing at one level will inherently impact the next level.

We will not be focusing on using algorithms to process whole documents or anything above that level, so we can look at preprocessing at the remaining levels: characters, words, sentences, and paragraphs. 

<b>For characters</b>, we typically process textual data by removing special characters, punctuations, and normalizations. However, this is not necessarily the case for every NLP approach. In some cases, taking away these special characters, punctuations, and normalizations will completely alter the meaning of higher levels of text.

<b>For words</b>, we first have to define words within our text snippets. This is known as segmenting: in the English language, we can use white space to segment text into words. However, this is not always the case with other languages, so it is important to know the linguistic rules of the primary language you are working with. Additionally, we perform some removals and/or replacements for odd words such as abbreviations, acronyms, numbers, and misspelled words. Finally, we normalize our text data; this is typically done by stemming (covered later in this section) or lemmatization.

<b>For sentences</b>, we first have to define sentence boundaries. In most languages, including English, sentences are started with a capital letter and ended with a period. However, periods and capital letters are also used within sentences, and captial letters or periods might not even exist in other languages, again stressing the importance of knowing the language's rules.  Within sentences, we can also mark phrases. In English sentences, we can typically label phrases of sentences with a subject, predicate, and object; the subject is the one performing the action, or predicate, on or directed towards the object. Finally, we parse the sentence, or tag the words with their respective part of speech. Note that this can only be done on the sentence level, not the word level, as the same word can have a different part of speech depending on how it is used in a sentence.

Finally, <b>for paragraphs</b>, our primary level of preprocessing is to understand the text by extracting the meaning of the text. This can include sentiments, emotions, intentions, etc. A good way of doing this is to perform abstractive summarization, or constructing new text that is similar in meaning to the old text. 


The actual process of text preprocessing, however, does not necessarily flow distinctly from one level to another. The order of preprocessing steps, as well as whether or not to include specific steps, depends largely on the application. An example process is given here: 
<ol>
<li>Segmentation: breaking text into sentences</li>
<li>Spelling correction</li>
<li>Noise removal: includes text that would otherwise confuse the main text, including emojis, foreign language, and http links</li>
<li>Language detection: identifying the language used in a body of text</li>
<li>Stop-words removal: these words are typically high frequency, generic, and less context specific
change cases</li>
<li>Lemmatization</li>
<li>Tokenization: break texts into words, phrases, symbols, and other semantically useful units, meaningful elements</li>
<li>Parsing: part of speech identification</li>
<li>Standardization</li>
<li>Stemming: reduces tokens to base forms (stems), removes affixes</li>
</ol>

Many of these steps may be unfamiliar - that is perfectly fine, as they will be covered later in this section. 

It is commonly accepted that preprocessing text helps improve accuracy, so it is critical to perform good, appropriate preprocessing.



## 1.3 Procedures

#### 1.3.1 Stemming 

Developed by Martin Porter in 1980, the <b>Porter Stemmer</b> is the oldest and most commonly used stemming algorithm in many languages. It is slightly time consuming, so another stemming algorithm that can be used is the Lancaster stemmer. This stemmer is very aggressive, however, and is especially harmful for short tokens, where the token itself may become unclear or altered to a point where it loses meaning.

As mentioned previously, stemming is performed to help normalize our textual data; this is done so that we do not end up with many different words with very similar meaning. For example, stemming turns "watch", "watched", "watching", and "watches" into "watch". To do this, the algorithm accesses a database where known word forms, such as the various forms of "watch", are grouped together, then analyzes the text for all instances of these forms and turns them all into the root word. Stemmers also typically perform preprocessing tasks like turning words not at the beginning of a sentence into lowercase and removing stop words, which are common words like "the" and "that" which show up in a variety of contexts and typically do not change meaning across these contexts.

However, there are always irregularities when using any stemming algorithm. Take the following examples: 
<ul>
<li>"I bought an Apple" vs. "I bought an apple": Here, the stemming algorithm removed the capital on "Apple", but we can clearly see that this drastically altered the meaning; now, the speaker has bought a fruit instead of a product from the company Apple.  </li>
<li>"I watched <i>The Who</i>" => remove stop words => "I watched": Here, the stemming algorithm removed the stop words "the" and "who", but this again changed the meaning of the sentence. Here, instead of watching the TV show <i>The Who</i>, the speaker has now watched something unspecified.</li> 
<li>"Take that!" => remove stop words => "Take": Like the above example, the stemming algorithm removed a stop word, "that". However, the sentence now loses all meaning: it is no longer a complete sentence, and without the "that" which served to add a violent, fighting intent to the original sentence, this emotion and intent is completely gone.
</ul>

Let's try these stemmers, as well as a third type, the Snowball Stemmer, on some tokens:

In [ ]:
from nltk import stem

tokens =  ['narrative', 'comprehensives', 'publicised', 'moderately', 'prosperous', 'legitimizes', 'according', 'glorifies'] 

porter = stem.porter.PorterStemmer() # Porter stemmer
snowball = stem.snowball.EnglishStemmer() # Snowball stemmer
lancaster = stem.lancaster.LancasterStemmer() # Lancaster stemmer

print('Porter Stemmer:', [porter.stem(i) for i in tokens])
print('Snowball Stemmer:', [snowball.stem(i) for i in tokens])
print('Lancaster Stemmer:', [lancaster.stem(i) for i in tokens])

<b>Exercise 1.5</b>: How do the different stemmers compare for the list of words given to it in the above example? Which one might have the most trouble on a larger set of tokens>

<b>Exercise 1.6</b>: Modify the above code to stem your own tokens. What words seem to be removed as stop words? How are words commonly shortened, and what types of words seem to cause the most trouble for the stemmers?

#### 1.3.2 Translation

Language detection is easy using pre-existing algorithms, including Google Translate, which can be downloaded and installed for use in Python with the following:

In [ ]:
conda install fasttext
pip  install googletrans==3.1.0a0
conda install -c conda-forge langdetect 
conda install -c conda-forge spacy_langdetect


To use the translation service, we can run the following code to import and load the model, then use it to predict languages:

In [ ]:
import fasttext
model = fasttext.load_model('lid.176.ftz')
print(model.predict('الشمس تشرق', k=2))  # top 2 matching languages
print(model.predict("影響包含對氣候的變化以及自然資源的枯竭程度",k=2))  # top 2 matching languages
print(model.predict(t1, k=3))
print(model.predict('四个全面', k=3))

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
detect('四个和尚没水喝'), detect('今一はお前さん')

from googletrans import Translator
t = Translator().detect("四个和尚")
t.lang, t.confidence

We can also use $\texttt{langdetect}$ to detect languages as follows:

In [ ]:
from langdetect import detect
print(detect("War doesn't show who's right, just who's left."))
print(detect("Ein, zwei, drei, vier"))
print(detect('四个和尚没水喝'))
print(detect('今一はお前さん'))

<b>Exercise 1.5:</b> Run the previous lines of code.
<ol>
<li>What are the matching languages for the provided example sentences?</li>
<li>How confident is the translator in the predicted language for the final sentence?</li>
</ol>

Now that we have an algorithm for predicting language, let's try adding language as a column to our UN SDG dataset.

In [ ]:
text_df["lang"] = text_df["text"].apply(lambda x: detect(x))

<b>Exercise 1.6</b>: Use $\texttt{text\_df.lang.value\_counts()}$ to count the number of texts for each language in our UN SDG dataset thus far. How many texts are there for each language?

Our dataset includes some non-English texts, which is understandable: while English is a widely spoken language and is the dominant language on the Internet, there are thousands of other languages. However, English is the most common language in the dataset, so we should consider: do we need to remove the non-English texts?

<b>Exercise 1.7</b>: Use $\texttt{text\_df.query("lang == 'fr'")["text"]}$ to output the texts written in French. Examine the output of these sentences. Does it make sense to remove them? If so, write the necessary code to do so.

#### 1.3.3 Parts of Speech

In addition to splitting sentences and texts into tokens, it is also important to add the appropriate part of speech to tokens within the text. 

For the example sentence “The dog ate the bone”, we can first split the sentence into a noun phrase (NP), "the dog", and a verb phrase (VP), "ate the bone". The first NP can be split into a determinant (D) "the", and a noun (N) "dog". The VP can be split into a verb "ate", and another NP "the bone", which then gest split in a similar manner. Hence was have labeled every word with its appropriate part of speech.

<b>Exercise 1.8:</b> Perform this similar breakdown and labeling with the following sentences. 
<ol>
<li>“This is a sentence.”</li>
<li>"Er lebt mit seinen Eltern und seiner Schwester in Berlin." (use a translator for help)</li>
<li>“In China today, President Xi Jinping’s new grand narrative is framed by the widely publicised ‘four comprehensives’.”</li>
</ol>

The library $\texttt{spaCy}$ is used to tag parts of speech via computer:


In [ ]:
import spacy
import en_core_web_sm
from spacy_langdetect import LanguageDetector
nlp = spacy.load("en_core_web_sm")
nlp = en_core_web_sm.load()
doc = nlp("This is a sentence.")
print([(w.text, w.pos_) for w in doc])
text_content = "Er lebt mit seinen Eltern und seiner Schwester in Berlin."
doc = nlp(text_content) 
print([(w.text, w.pos_) for w in doc])
doc = nlp(t1)
print([(w.text, w.pos_) for w in doc])

<b>Exercise 1.9</b>: Modify the above code to tag the parts of speech for the last sentence in Exercise 1.y. 

#### 1.3.4 Tokenizers

The primary goal of a tokenizer is to break text into tokens. This is highly language dependent; in English, we use white space and punctuations to as word and token separators, but this may not work in some languages. 

For the purposes of building ML models, there are several things that tokenizers track in addition to creating the tokens themselves. Tokenizers track <i>features</i>, or the frequency of occurrence of each individual token (normalized or not). They also produce a <i>sample</i>, or a vector of all the token frequencies for a given document. 

When a tokenizer vectorizes a corpus, it counts the occurrences of tokens in each document, normalizes and weights these occurrences, and constructs an n-gram, which is discussed in the next section.



#### 1.3.5 Named Entity Recognition

The purpose of named entity recognition (NER) is self-explanatory; it extracts entities from data.

<b>Exercise 1.10:</b> Try the Allen NLP NER demo found at https://demo.allennlp.org/named-entity-recognition/named-entity-recognition. 
<ol>
<li>Use the below sentence in the demo. Which phrases are labeled as named entities? </li>

'The Israel Oceanographic and Limnological Research station monitors the quantity and quality of water along the coastline of the Mediterranean Sea. The Nature and Parks Authority (NPA) monitors water quality in rivers on behalf of the MoEP. Mekorot and local authorities monitor drinking water quality under the supervision of the Ministry of Health. The Ministry of Health monitors effluent quality prior to its use in the agricultural sector.'

<li>Now use text.lower() to make the entire sentence lowercase and use that as an input into the Allen NLP NER. Which phrases are now labeled as named entities?</li>

'the israel oceanographic and limnological research station monitors the quantity and quality of water along the coastline of the mediterranean sea. the nature and parks authority (npa) monitors water quality in rivers on behalf of the moep. mekorot and local authorities monitor drinking water quality under the supervision of the ministry of health. the ministry of health monitors effluent quality prior to its use in the agricultural sector.'

</ol>


#### 1.3.6 String Operations

Now that we know about various preprocessing techniques, we can review some basic string operations. These will also be important in identifying various sentences and tokens within our dataset in case we would like to focus on specific words, or it can help in identifying cases, numeric characters, and other tests that we might need to conduct. A few are given below:

In [ ]:
s.startswith(t) #test if s starts with t
s.endswith(t) #test if s ends with t
t in s # test if t is a substring of s
s.islower() # test if s contains cased characters and all are lowercase
s.isupper() # test if s contains cased characters and all are uppercase
s.isalpha() # test if s is non-empty and all characters in s are alphabetic
s.isalnum() # test if s is non-empty and all characters in s are alphanumeric
s.isdigit() # test if s is non-empty and all characters in s are digits
s.istitle() # test if s contains cased characters and is titlecased (i.e. all words in s have initial capitals)


As an example to work through, let's take the Chinese phrase '四个全面' and see if we can locate it in the texts. This phrase, anglicized as "sigequanmian" and roughly translating to "four comprehensives", was publicized by Chinese President Xi Jinping as a set of goals for China.

In [ ]:
text_df[text_df["text"].apply(lambda x: x.find('四个全面')) > 0]["text"]

One possible string operation is comparison, as given below:

In [ ]:
t1 = text_df[text_df["text"].apply(lambda x: x.find('四个全面')) > 0]["text"].values[0]
print(t1 == text_df.iloc[18009].text)
print(t1 is text_df.iloc[18009].text)
print(t1 == text_df.iloc[18009].text.casefold())
print(t1.lower() == text_df.iloc[18009].text.casefold())

We can see that Python takes uses strict comparisons, meaning that the two sentences will not be equal unless the cases (capital and lowercase) are matched.

Note that in the above example, the $\texttt{.lower()}$ and $\texttt{.casefold()}$ yielded equal results. However, this is not the case for all languages. Take the German letter ß:

In [ ]:
text = 'groß'

# convert text to lowercase using casefold()
print('Using casefold():', text.casefold()) 

# convert text to lowercase using lower()
print('Using lower():', text.lower())

The German word "groß" translates as large or big, and this meaning is kept using $\texttt{.lower()}$. However, when using $\texttt{.casefold()}$, it becomes "gross", which is used in English to indicate disgust. It is up to you to decide which of these to use when converting to lowercase; however, it is best to keep this consistent across all your code so as to avoid potential problems such as this.

### 1.4 More Exercises